In [1]:

############################################## 00. 필요한 파이썬 라이브러리 불러오기 #####################################################
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import webbrowser

import sys
import plotly.express as px
from numpy import random
import missingno

from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier

import spark
import pydot 
from IPython.display import Image
import pyarrow as pa

import urllib
import math
from tqdm import tqdm

################### 설정 ########################
pd.options.display.float_format = '{:.10f}'.format

# 데이터 불러오기

### 자전거 대여/반납이력 정보

In [2]:
# ### 월별 따릉이 데이터를 병합 (08월~12월)
# bike_df = pd.DataFrame()

# month_list = ["08","09","10","11","12"]
# for i in month_list:
#     data = pd.read_csv("공공자전거 대여이력 정보_21"+ i +".csv",encoding='cp949')
#     bike_df = pd.concat((bike_df,data),sort=False)
#     print(data.head())


> 데이터 용량이 너무 커서 가장 빠른 처리속도를 보이는 spark 라이브러리의 parque 형식 사용

In [3]:
# 데이터프레임의 스키마 정보를 유지하면서 parquet 파일로 저장

# file_name = 'bike_rental.parque'
# bike_df.to_parquet(file_name)

In [4]:
# parquet 데이터 불러오기
bike_df = pd.read_parquet('bike_rental.parque')
# bike_df = spark.read.format('parquet').load(file_name)

In [5]:
#bike_df
bike_df.info()
bike_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13813512 entries, 0 to 1704296
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   자전거번호     object 
 1   대여일시      object 
 2   대여 대여소번호  int64  
 3   대여 대여소명   object 
 4   대여거치대     int64  
 5   반납일시      object 
 6   반납대여소번호   int64  
 7   반납대여소명    object 
 8   반납거치대     int64  
 9   이용시간      int64  
 10  이용거리      float64
dtypes: float64(1), int64(5), object(5)
memory usage: 1.2+ GB


,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-32016,2021-08-07 22:01:16,3,중랑센터,0,2021-08-07 22:07:10,646,장한평역 1번출구 (국민은행앞),0,5,1239.8900000000
1,SPB-55081,2021-08-08 22:00:59,3,중랑센터,0,2021-08-08 22:22:27,4819,면목동 새싹어린이공원 앞,0,21,4210.0000000000
2,SPB-32688,2021-08-21 22:01:24,3,중랑센터,0,2021-08-21 22:11:42,540,군자역 7번출구 베스트샵 앞,0,10,2269.7100000000
3,SPB-45676,2021-08-22 21:59:53,3,중랑센터,0,2021-08-22 22:13:12,540,군자역 7번출구 베스트샵 앞,0,13,2420.0000000000
4,SPB-33811,2021-08-27 22:00:45,3,중랑센터,0,2021-08-27 22:11:09,540,군자역 7번출구 베스트샵 앞,0,10,2335.2900000000


> 대여거치대, 반납거치대에 있는 번호의 의미?


In [6]:
bike_df["대여거치대"][bike_df["대여거치대"]!=0]
bike_df["대여거치대"].unique()
bike_df.groupby(by = ['대여거치대', '반납거치대']).count()

자전거번호      대여일시  대여 대여소번호   대여 대여소명      반납일시   반납대여소번호  \
대여거치대 반납거치대                                                               
0     0      13512659  13512659  13512659  13512659  13512659  13512659   
      99          406       406       406       406       406       406   
4     2             1         1         1         1         1         1   
9     4             1         1         1         1         1         1   
99    0          1077      1077      1077      1077      1077      1077   
      99       299368    299368    299368    299368    299368    299368   

               반납대여소명      이용시간      이용거리  
대여거치대 반납거치대                                
0     0      13512659  13512659  13512659  
      99          406       406       406  
4     2             1         1         1  
9     4             1         1         1  
99    0          1077      1077      1077  
      99       299368    299368    299368

- 0>0이 대부분이고, 99>99 혹은 99>0이 존재

### 자전거 대여소 정보 

> 불러오기

In [7]:
station_df = pd.read_csv("공공자전거 대여소 정보(21.12월 기준).csv",encoding='cp949')
station_df = station_df.iloc[4:,]
station_df.columns = ["대여소번호","보관소(대여소명)","소재지","주소","위도","경도","설치시기","거치대수-LCD","거치대수-QR","운영방식"]

station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2586 entries, 4 to 2589
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   대여소번호      2586 non-null   float64
 1   보관소(대여소명)  2586 non-null   object 
 2   소재지        2586 non-null   object 
 3   주소         2586 non-null   object 
 4   위도         2586 non-null   object 
 5   경도         2586 non-null   object 
 6   설치시기       2586 non-null   object 
 7   거치대수-LCD   1400 non-null   object 
 8   거치대수-QR    1187 non-null   object 
 9   운영방식       2586 non-null   object 
dtypes: float64(1), object(9)
memory usage: 202.2+ KB


In [8]:
station_df.head(10)

,대여소번호,보관소(대여소명),소재지,주소,위도,경도,설치시기,거치대수-LCD,거치대수-QR,운영방식
4,102.0000000000,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488,126.9106293,2015-09-06,20,NaN,LCD
5,103.0000000000,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.55495071,126.9108353,2015-09-06,14,NaN,LCD
6,104.0000000000,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.55062866,126.9149857,2015-09-06,13,NaN,LCD
7,105.0000000000,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.55000687,126.9148254,2015-09-06,5,NaN,LCD
8,106.0000000000,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.54864502,126.9128265,2015-09-06,10,NaN,LCD
9,107.0000000000,신한은행 서교동금융센터점 앞,마포구,서울특별시 마포구 월드컵북로 35,37.55751038,126.9185028,2015-09-06,5,NaN,LCD
10,108.0000000000,서교동 사거리,마포구,서울특별시 마포구 양화로 93,37.55274582,126.9186173,2015-09-06,10,NaN,LCD
11,109.0000000000,제일빌딩 앞,마포구,서울특별시 마포구 어울마당로 26,37.54769135,126.9199829,2015-09-07,10,NaN,LCD
12,111.0000000000,상수역 2번출구 앞,마포구,서울특별시 마포구 와우산로 40,37.54787064,126.9235306,2015-09-07,10,NaN,LCD
13,112.0000000000,극동방송국 앞,마포구,서울특별시 마포구 와우산로 56,37.54920197,126.9232025,2015-09-07,10,NaN,LCD


### 데이터 타입 변경
- 추후 칼럼별 계산 시 object 타입을 변경해주어야 함

In [9]:
station_df[["거치대수-LCD","거치대수-QR"]] = station_df[["거치대수-LCD","거치대수-QR"]].fillna(0)

station_df = station_df.astype({"대여소번호":'int64',"위도":'float64',"경도":'float64',"거치대수-LCD":'int64',"거치대수-QR":'int64'})

station_df

,대여소번호,보관소(대여소명),소재지,주소,위도,경도,설치시기,거치대수-LCD,거치대수-QR,운영방식
4,102,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488000,126.9106293000,2015-09-06,20,0,LCD
5,103,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.5549507100,126.9108353000,2015-09-06,14,0,LCD
6,104,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.5506286600,126.9149857000,2015-09-06,13,0,LCD
7,105,합정역 5번출구 앞,마포구,서울특별시 마포구 양화로 48,37.5500068700,126.9148254000,2015-09-06,5,0,LCD
8,106,합정역 7번출구 앞,마포구,서울특별시 마포구 독막로 4,37.5486450200,126.9128265000,2015-09-06,10,0,LCD
...,...,...,...,...,...,...,...,...,...,...
2585,5070,마곡13단지힐스테이트마스터 후문(1320동 앞),강서구,서울시 강서구 마곡중앙1로 71 마곡13단지힐스테이트마스터 후문 인근,37.5567627000,126.8186951000,2021-11-11,0,15,QR
2586,5072,김포공항입구 교통섬,강서구,공항동 74-5,37.5615463300,126.8070831000,2021-09-28,0,10,QR
2587,5073,우장산역 헤링턴타워 앞,강서구,화곡동 1013-3,37.5466194200,126.8362351000,2021-10-20,0,6,QR
2588,5074,마곡동 주니어마곡점앞,강서구,강서구 마곡동 798-17,37.5582351700,126.8257599000,2021-11-19,0,10,QR


# 결측값 , 중복값 확인


In [10]:
def missing_table(df):
    df = df.isnull().sum().reset_index()
    df.columns = ['column_name', 'missing_count']
    df = df.loc[df['missing_count']>0]
    df = df.sort_values(by='missing_count')
    return df

In [11]:
display(missing_table(bike_df))
display(missing_table(station_df))

,column_name,missing_count


,column_name,missing_count


> 두 데이터 모두 결측값 없음

> Q : 렌탈데이터의 역 갯수와 역데이터의 역 갯수가 다르다.
- 두 데이터가 완벽하게 매칭되진 않음을 시사

In [12]:
num_station_in_rental_data = len(bike_df["대여 대여소명"].unique())
num_station_in_station_data = len(station_df["보관소(대여소명)"].unique())

print("렌탈데이터 역 갯수:",num_station_in_rental_data,"역데이터 역 갯수:",num_station_in_station_data)

렌탈데이터 역 갯수: 2476 역데이터 역 갯수: 2584


> 역데이터에서 중복되는 대여소명은 2개이다.

In [13]:
duplicated_station_df = station_df[station_df["보관소(대여소명)"].duplicated()]

display(duplicated_station_df)

,대여소번호,보관소(대여소명),소재지,주소,위도,경도,설치시기,거치대수-LCD,거치대수-QR,운영방식
2152,3810,신봉천주유소,관악구,남부순환로 1880,37.4788818400,126.9584198000,2021-02-01,0,10,QR
2514,4811,한양수자인아파트 앞,중랑구,면목동 1085-1,37.5803222700,127.0941544000,2021-02-17,0,10,QR


- 신봉천주유소, 한양수자인아파트 두 데이터는 역데이터에 중복으로 존재

In [14]:
def find_duplicated_row(df):
    duplicated_df = df[station_df["보관소(대여소명)"].duplicated()]
    isin_con = df['보관소(대여소명)'].isin(duplicated_df['보관소(대여소명)'])
    return df[isin_con]

find_duplicated_row(station_df)

,대여소번호,보관소(대여소명),소재지,주소,위도,경도,설치시기,거치대수-LCD,거치대수-QR,운영방식
1283,1828,한양수자인아파트 앞,금천구,서울특별시 금천구 시흥대로150길 6,37.4753189100,126.8988876000,2017-05-08,10,0,LCD
1501,2199,신봉천주유소,관악구,남부순환로 1884,37.4787712100,126.9586945000,2020-04-30,0,8,QR
2152,3810,신봉천주유소,관악구,남부순환로 1880,37.4788818400,126.9584198000,2021-02-01,0,10,QR
2514,4811,한양수자인아파트 앞,중랑구,면목동 1085-1,37.5803222700,127.0941544000,2021-02-17,0,10,QR


- 대여소 이름은 같지만 대여소번호가 다르니 다른 위치의 대여소명이라고 할 수 있을 듯.

## Q : 두 데이터를 연결 시킬 수 없을끼?
- bike_df와 station_df를 하나의 데이터프레임으로 합치고 싶다.
- 대여소번호로 이어볼 수 있지 않을까?

In [15]:
display(bike_df[bike_df["대여 대여소번호"]==102].head())
display(station_df[station_df["대여소번호"]==102])

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
21,SPB-45127,2021-08-01 00:57:08,102,망원역 1번출구 앞,0,2021-08-01 01:07:14,113,홍대입구역 2번출구 앞,0,10,0.0000000000
22,SPB-48183,2021-08-01 00:37:42,102,망원역 1번출구 앞,0,2021-08-01 01:14:22,107,신한은행 서교동금융센터점 앞,0,36,4540.0000000000
23,SPB-35457,2021-08-01 02:13:19,102,망원역 1번출구 앞,0,2021-08-01 02:24:55,112,극동방송국 앞,0,11,1731.8800000000
24,SPB-53028,2021-08-01 01:56:13,102,망원역 1번출구 앞,0,2021-08-01 02:40:46,102,망원역 1번출구 앞,0,44,0.0000000000
25,SPB-53028,2021-08-01 02:41:25,102,망원역 1번출구 앞,0,2021-08-01 03:01:44,195,모래내고가차도,0,20,2334.1900000000


,대여소번호,보관소(대여소명),소재지,주소,위도,경도,설치시기,거치대수-LCD,거치대수-QR,운영방식
4,102,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.5556488000,126.9106293000,2015-09-06,20,0,LCD


In [16]:
min_station_num = station_df["대여소번호"].min()
max_station_num = station_df["대여소번호"].max()

print(f'대여소번호는 {min_station_num}번 부터 {max_station_num} 까지 입니다.')


대여소번호는 102번 부터 5075 까지 입니다.


### 자전거의 총 대수는?

- 자전거 번호 기준으로 groupby

In [17]:
groupby_bike_ID_df = bike_df.groupby("자전거번호").count()

print("렌탈 데이터의 총 자전거 댓수는 :",len(groupby_bike_ID_df))

렌탈 데이터의 총 자전거 댓수는 : 30277


### 매칭이 되지 않는 역들에 대한 고찰

-대여소 이름으로 두 데이터를 매칭시켜보자.

In [18]:
index = bike_df["대여 대여소명"].isin(station_df["보관소(대여소명)"])
not_index = [not i for i in index]

unknown_station_rental_df = bike_df[not_index]

print("역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터의 row의 갯수:",len(unknown_station_rental_df))
display("역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터의 rows:",unknown_station_rental_df)
print("역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터 역의 갯수:",len(unknown_station_rental_df["대여 대여소명"].unique()))
print("역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터의 역의 이름:",unknown_station_rental_df["대여 대여소명"].unique())

역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터의 row의 갯수: 513931


'역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터의 rows:'

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-32016,2021-08-07 22:01:16,3,중랑센터,0,2021-08-07 22:07:10,646,장한평역 1번출구 (국민은행앞),0,5,1239.8900000000
1,SPB-55081,2021-08-08 22:00:59,3,중랑센터,0,2021-08-08 22:22:27,4819,면목동 새싹어린이공원 앞,0,21,4210.0000000000
2,SPB-32688,2021-08-21 22:01:24,3,중랑센터,0,2021-08-21 22:11:42,540,군자역 7번출구 베스트샵 앞,0,10,2269.7100000000
3,SPB-45676,2021-08-22 21:59:53,3,중랑센터,0,2021-08-22 22:13:12,540,군자역 7번출구 베스트샵 앞,0,13,2420.0000000000
4,SPB-33811,2021-08-27 22:00:45,3,중랑센터,0,2021-08-27 22:11:09,540,군자역 7번출구 베스트샵 앞,0,10,2335.2900000000
...,...,...,...,...,...,...,...,...,...,...,...
1704292,SPB-51882,2021-12-30 09:39:45,9999,상담센터,0,2021-12-30 09:52:44,346,맥스타일 앞,0,12,0.0000000000
1704293,SPB-51134,2021-12-30 18:55:31,9999,상담센터,0,2021-12-30 18:59:04,1986,태평양물산빌딩,0,3,0.0000000000
1704294,SPB-37732,2021-12-30 21:12:40,9999,상담센터,0,2021-12-30 21:15:54,188,홍은동 정원여중 입구,0,3,345.4500000000
1704295,SPB-51854,2021-12-22 09:46:16,88888,영남배송,0,2021-12-22 10:03:24,236,문래동자이아파트 앞,0,17,0.0000000000


역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터 역의 갯수: 132
역데이터의 대여소명으로 매칭이 되지 않는 렌탈 데이터의 역의 이름: ['중랑센터' '상암센터 정비실' '위트콤' '마포구민체육센터 앞' '독립문 어린이 공원' '신길삼거리(우리은행)'
 '다산 어린이공원' 'DDP 패션몰' '예수소망교회' '서울역5번출구' '용답초등학교' '청계한신휴플러스앞 삼거리'
 '성복교회 앞 (장평교 아래)' '신정3동주민센터' '목동1단지아파트 118동 앞' '목동1단지아파트 상가 앞'
 '등촌역 4번 출구' '국방부민원실옆' '삼각지12번 출구' '삼각지 자이아파트' 'KCC용산월츠타워' '현대 안성타워'
 '디지털미디어 시티역 4번출구(DMC역)' '은평뉴타운 은뜨락아파트앞' '강동구평생학습관앞' '올림픽로 836(아리수로)'
 'LG베스트샵 종암점' '래미안장위포레카운티(102동)' '래미안장위퍼스트하이(513동)' '구립망우청소년독서실'
 '면목 홈플러스 2' '용마지구대 옆' '419민주묘지역' '솔샘역 2번 출구' '419탑 앞' '419민주묘지역 1번출구'
 '419민주묘지 입구사거리' '북서울 꿈의 숲 입구' '상계34동 주민센터 대여소' '육군사관학교  앞' '월계미륭아파트 정문'
 '상계보람아파트 202동' '다이소방학점 앞' '피자헛 앞(도봉병원 옆)' 'ESA아파트 상가 앞' '창포원 남쪽 입구'
 '창원초등학교 교차로' '창동 주공 4단지 입구 옆' '창동시장입구 사거리' 'LG전자 베스트샾 대리점 앞' '숭실대학교(중문)'
 '삼성서비스센터(관악점)' '대학동주민센터' '강남세브란스교차로  래미안그레이튼 104동 앞' '대청역 1번출구  뒤'
 '르네상스호텔사거리 역삼지하보도 3번출구 앞 (더 오번 빌딩 명동칼국수 앞)' '압구정역 교차로'
 '역삼서초삼성 세무서 앞 (역삼빌딩 앞)' '선정릉 디캠프(DCAMP)앞' 'KT반포지사' '신흥교(애견훈련소)' '신논현타워 앞'
 '복정역 2번출구 후문 (장지치안센터)' '석촌고분역 4번출구'

In [19]:
indexing = bike_df["반납대여소명"].isin(station_df["보관소(대여소명)"])
not_indexing = [not i for i in indexing]

unknown_station_return_df = bike_df[not_indexing]

print("역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터의 row의 갯수:",len(unknown_station_return_df))
display("역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터의 rows:",unknown_station_return_df)
print("역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터 역의 갯수:",len(unknown_station_return_df["반납대여소명"].unique()))
print("역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터의 역의 이름:",unknown_station_return_df["반납대여소명"].unique())

역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터의 row의 갯수: 506509


'역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터의 rows:'

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
10,SPB-34033,2021-08-24 15:52:18,5,상암센터 정비실,0,2021-08-24 15:59:19,5,상암센터 정비실,0,7,0.0000000000
14,SPB-30046,2021-08-10 10:11:27,10,위트콤,0,2021-08-10 10:15:29,10,위트콤,0,4,0.0000000000
15,SPB-30046,2021-08-11 15:41:56,10,위트콤,0,2021-08-11 15:45:39,10,위트콤,0,3,0.0000000000
16,SPB-30084,2021-08-17 15:45:21,10,위트콤,0,2021-08-17 15:48:04,10,위트콤,0,2,0.0000000000
17,SPB-30084,2021-08-17 15:49:05,10,위트콤,0,2021-08-17 16:57:26,10,위트콤,0,68,0.0000000000
...,...,...,...,...,...,...,...,...,...,...,...
1704266,SPB-43098,2021-12-20 09:58:32,9999,상담센터,0,2021-12-20 10:02:11,9999,상담센터,0,3,0.0000000000
1704270,SPB-43098,2021-12-21 15:51:12,9999,상담센터,0,2021-12-21 16:29:45,9999,상담센터,0,38,0.0000000000
1704271,SPB-43098,2021-12-21 17:15:57,9999,상담센터,0,2021-12-21 17:26:18,9999,상담센터,0,10,0.0000000000
1704279,SPB-43098,2021-12-24 09:00:00,9999,상담센터,0,2021-12-24 09:03:29,9980,에이텍,0,3,0.0000000000


역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터 역의 갯수: 130
역데이터의 대여소명으로 매칭이 되지 않는 반납 데이터의 역의 이름: ['상암센터 정비실' '위트콤' '마포구민체육센터 앞' '벽산상암스마트큐브' '디지털미디어 시티역 4번출구(DMC역)'
 '신사두산위브2차아파트 앞' '목동1단지아파트 상가 앞' '어위쉬예다인오피스텔' '상암월드컵파크5단지(상암중학교)'
 '은평뉴타운 은뜨락아파트앞' '삼성서비스센터(관악점)' '그랜드아이파크아파트 앞' '서울지방고용노동청 앞' '김영혜손뜨개 앞'
 '신길삼거리(우리은행)' '신논현타워 앞' '목동1단지아파트 118동 앞' '래미안장위퍼스트하이(513동)' '등촌역 4번 출구'
 '마곡나루역 2번 출구' '래미안신공덕3차아파트' 'KCC용산월츠타워' '삼각지 자이아파트' '국회6문' '독립문 어린이 공원'
 '서울도시가스 앞' '명동역9번 출구' '프런트원(공덕역4번출구 앞)' '흑석동효사정앞' '등나무 근린공원(시립북서울미술관 앞)'
 'LG베스트샵 종암점' '문정동 힐스데이트 에코 송파' '다산 어린이공원' 'ESA아파트 상가 앞' '용답초등학교' '서울이대병원'
 '숭실대학교(중문)' '국방부민원실옆' '국회소통관' '석계역 3번 출구' '신도림7차e편한세상아파트 806동 앞'
 'SK리쳄블(상수도사업본부)' '신이문빗물펌프장 옆' '성복교회 앞 (장평교 아래)' '태극당' '서울역5번출구'
 'LG전자 베스트샾 대리점 앞' '강서세무서 앞' '삼각지12번 출구' '올림픽로 836(아리수로)' '대학동주민센터'
 '현대 안성타워' '르네상스호텔사거리 역삼지하보도 3번출구 앞 (더 오번 빌딩 명동칼국수 앞)' '광운대학교 중앙도서관'
 '복정역 2번출구 후문 (장지치안센터)' 'DDP 패션몰' '압구정역 교차로' '수명산파크 4단지 411동 앞'
 '한남 준 JFSS 앞' '국회2문' '청계한신휴플러스앞 삼거리' '대청역 1번출구  뒤' 'KT반포지사' '양남사거리1'
 '래미안에스티움 앞 (신풍로)'

# 대여소 번호에 따른 분석

In [20]:
# 역데이터에 존재하는 역 번호 리스트
station_num_list = station_df["대여소번호"]

# 렌탈/반납 데이터의 역 번호가 역데이터의 역번호와 매칭되는 경우
isin_station_bike_df= bike_df[(bike_df["대여 대여소번호"].isin(station_num_list.unique()))&bike_df["반납대여소번호"].isin(station_num_list.unique())]

#역데이터의 대여소 번호로 매칭되지 않는 렌탈/반납 데이터. 즉 대여/반납이 이상한 곳으로 간 리스트
N_isin_station_bike_df =  bike_df[(~bike_df["대여 대여소번호"].isin(station_num_list.unique()))|(~bike_df["반납대여소번호"].isin(station_num_list.unique()))]
display(N_isin_station_bike_df)


,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-32016,2021-08-07 22:01:16,3,중랑센터,0,2021-08-07 22:07:10,646,장한평역 1번출구 (국민은행앞),0,5,1239.8900000000
1,SPB-55081,2021-08-08 22:00:59,3,중랑센터,0,2021-08-08 22:22:27,4819,면목동 새싹어린이공원 앞,0,21,4210.0000000000
2,SPB-32688,2021-08-21 22:01:24,3,중랑센터,0,2021-08-21 22:11:42,540,군자역 7번출구 베스트샵 앞,0,10,2269.7100000000
3,SPB-45676,2021-08-22 21:59:53,3,중랑센터,0,2021-08-22 22:13:12,540,군자역 7번출구 베스트샵 앞,0,13,2420.0000000000
4,SPB-33811,2021-08-27 22:00:45,3,중랑센터,0,2021-08-27 22:11:09,540,군자역 7번출구 베스트샵 앞,0,10,2335.2900000000
...,...,...,...,...,...,...,...,...,...,...,...
1704292,SPB-51882,2021-12-30 09:39:45,9999,상담센터,0,2021-12-30 09:52:44,346,맥스타일 앞,0,12,0.0000000000
1704293,SPB-51134,2021-12-30 18:55:31,9999,상담센터,0,2021-12-30 18:59:04,1986,태평양물산빌딩,0,3,0.0000000000
1704294,SPB-37732,2021-12-30 21:12:40,9999,상담센터,0,2021-12-30 21:15:54,188,홍은동 정원여중 입구,0,3,345.4500000000
1704295,SPB-51854,2021-12-22 09:46:16,88888,영남배송,0,2021-12-22 10:03:24,236,문래동자이아파트 앞,0,17,0.0000000000


- 총 155633건. 아까보다는 생각보다 얼마 많지 않다!
- 즉 두 데이터 테이블 간의 매칭을 "대여소 명"이 아닌 "대여소번호"로 해야 한다.

### 누락된 대여소 번호
- 102~5075번 사이에 누락된 반납대여소번호가 있다?
- 수리센터가 아님에도 역정보데이터에 누락되어 있는 경우가 있나?

"대여 대여소번호"와 "반납대여소번호" 에서 각각 누락된 역 번호 확인

In [21]:
indexing_1rent = N_isin_station_bike_df["대여 대여소번호"].isin(range(min_station_num,max_station_num+1))
indexing_2rent = N_isin_station_bike_df["대여 대여소번호"].isin(station_num_list)

indexing_1return = N_isin_station_bike_df["반납대여소번호"].isin(range(min_station_num,max_station_num+1))
indexing_2return = N_isin_station_bike_df["반납대여소번호"].isin(station_num_list)


####
N_isin_station_bike_df[indexing_1rent]  #<  대여소번호가 102~5075까지만 있음
A = N_isin_station_bike_df[indexing_1rent & indexing_2rent]["반납대여소번호"].unique()
B = range(min_station_num,max_station_num)
C = np.intersect1d(A,B)
print(f'반납대여소번호 에서 누락된 역번호 목록은{C}이며 그 갯수는 {len(C)}입니다.')

#### 
N_isin_station_bike_df[indexing_1return]  #< 대여소번호가 102~5075까지만 있음
A = N_isin_station_bike_df[indexing_1return & indexing_2return]["대여 대여소번호"].unique()
B = range(min_station_num,max_station_num)
C = np.intersect1d(A,B)
print(f'대여 대여소번호 에서 누락된 역번호 목록은{C}이며 그 갯수는 {len(C)}입니다.')


####
missing_station_list = C
bike_df[bike_df["대여 대여소번호"].isin(missing_station_list)]


반납대여소번호 에서 누락된 역번호 목록은[ 152  158  257  375  482  581  675  707 1014 1320 1443 1545 1695 1700
 2361 2534 2607 2629 2905 3755 4038 4112 4704 4779]이며 그 갯수는 24입니다.
대여 대여소번호 에서 누락된 역번호 목록은[ 152  158  257  375  482  581  675  707 1014 1320 1443 1545 1695 1700
 2361 2534 2607 2629 2905 3755 4038 4112 4704 4779]이며 그 갯수는 24입니다.


,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
82886,SPB-32526,2021-08-01 00:09:56,152,마포구민체육센터 앞,0,2021-08-01 00:38:10,166,가재울 초등학교,0,28,4290.4300000000
82887,SPB-46806,2021-08-01 00:08:35,152,마포구민체육센터 앞,0,2021-08-01 00:38:29,166,가재울 초등학교,0,29,4050.0000000000
82888,SPB-41686,2021-08-01 00:09:15,152,마포구민체육센터 앞,0,2021-08-01 00:38:33,166,가재울 초등학교,0,29,4392.3000000000
82889,SPB-55955,2021-08-01 00:09:09,152,마포구민체육센터 앞,0,2021-08-01 01:04:35,4653,LG 베스트샵 앞,0,55,9680.0000000000
82890,SPB-53496,2021-08-01 00:09:18,152,마포구민체육센터 앞,0,2021-08-01 01:04:37,4653,LG 베스트샵 앞,0,55,10218.8200000000
...,...,...,...,...,...,...,...,...,...,...,...
1584154,SPB-52140,2021-12-17 08:40:14,4112,신이문빗물펌프장 옆,0,2021-12-17 08:48:18,1433,중화역 2번출구,0,8,1110.0000000000
1584155,SPB-48197,2021-12-17 20:48:54,4112,신이문빗물펌프장 옆,0,2021-12-17 20:55:19,4104,외대앞역 교차로,0,6,880.0000000000
1584156,SPB-56417,2021-12-18 07:31:08,4112,신이문빗물펌프장 옆,0,2021-12-18 07:48:32,1446,중랑전화국 교차로,0,17,3370.0000000000
1584157,SPB-31845,2021-12-19 12:47:18,4112,신이문빗물펌프장 옆,0,2021-12-19 13:00:45,4107,장안동근린공원 입구,0,13,0.0000000000


### 이용거리가 0인 데이터 분석

In [22]:
## 이용거리가 0인 데이터 도출하는 함수

def find_ditance_zero(df):
    df = df[df["이용거리"]==0]
    print("이동거리가 0인 데이터의 갯수 :",len(df),"개")
    return df
    

In [23]:
distance_zero_bike_df = find_ditance_zero(bike_df)

print("전체 데이터 중 이동거리가 0인 데이터의 비율 {:.2f}%".format(len(distance_zero_bike_df)/len(bike_df)*100))

이동거리가 0인 데이터의 갯수 : 2474536 개
전체 데이터 중 이동거리가 0인 데이터의 비율 17.91%


- 생각보다 이동거리가 0인 데이터가 많다.
    - 서울시에 문의한 결과, 측정이 잘못 된 데이터가 맞다고 한다.

### 고장 자전거 수리에 의한 이동

> rental/return raw data에는 자전거 수리점의 경우도 포함됨
> 어떻게 자전거 수리점을 추려낼 수 있을까?
- 대여소번호가 이상한 경우.
- 이름이 이상한 경우

- depot, 혹은 수리센터 이름으로 추정되는 사항들:
    
  -  '훈련원센터','상암센터 정비실' ,'위트콤','중랑센터','영남주차장 정비센터','에이텍','상담센터', '영남단말기정비','사대문센터' ,'_도봉센터','천호센터' ,'천왕센터' ,'영남배송' ,'개화정비','강남센터'

In [24]:
## 수리센터로 추정되는 대여소 이름 리스트
fix_station_list = ['훈련원센터','상암센터 정비실' ,'위트콤','중랑센터','영남주차장 정비센터','에이텍','상담센터', '영남단말기정비','사대문센터' ,'_도봉센터','천호센터' ,'천왕센터' ,'영남배송' ,'개화정비','강남센터p']

## bike_df의 OD에 수리센터 추정 이름이 있는 경우의 행만 인덱싱하여 출력
index = (bike_df["대여 대여소명"].isin(fix_station_list))|(bike_df["반납대여소명"].isin(fix_station_list))
fix_center_isin_OD_df = bike_df[index]

display(fix_center_isin_OD_df)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-32016,2021-08-07 22:01:16,3,중랑센터,0,2021-08-07 22:07:10,646,장한평역 1번출구 (국민은행앞),0,5,1239.8900000000
1,SPB-55081,2021-08-08 22:00:59,3,중랑센터,0,2021-08-08 22:22:27,4819,면목동 새싹어린이공원 앞,0,21,4210.0000000000
2,SPB-32688,2021-08-21 22:01:24,3,중랑센터,0,2021-08-21 22:11:42,540,군자역 7번출구 베스트샵 앞,0,10,2269.7100000000
3,SPB-45676,2021-08-22 21:59:53,3,중랑센터,0,2021-08-22 22:13:12,540,군자역 7번출구 베스트샵 앞,0,13,2420.0000000000
4,SPB-33811,2021-08-27 22:00:45,3,중랑센터,0,2021-08-27 22:11:09,540,군자역 7번출구 베스트샵 앞,0,10,2335.2900000000
...,...,...,...,...,...,...,...,...,...,...,...
1704292,SPB-51882,2021-12-30 09:39:45,9999,상담센터,0,2021-12-30 09:52:44,346,맥스타일 앞,0,12,0.0000000000
1704293,SPB-51134,2021-12-30 18:55:31,9999,상담센터,0,2021-12-30 18:59:04,1986,태평양물산빌딩,0,3,0.0000000000
1704294,SPB-37732,2021-12-30 21:12:40,9999,상담센터,0,2021-12-30 21:15:54,188,홍은동 정원여중 입구,0,3,345.4500000000
1704295,SPB-51854,2021-12-22 09:46:16,88888,영남배송,0,2021-12-22 10:03:24,236,문래동자이아파트 앞,0,17,0.0000000000


- 이동거리가 0인 row와 수리센터가 끼여있는 row끼리는 별 상관이 없다.

### geocoding5 practice

- 주소를 입력받아서 위,경도 좌표를 도출해주는 라이브러리 

### Directions 5

- 위도 경도를 입력받아서 최단거리를 포함한 정보를 도출해주는 라이브러리


In [25]:
############ 구형좌표계에서 두 지점의 위도 경도값을 받아서 직선거리 도출하는 함수 
def distance(lat1, lng1, lat2, lng2):

    lat1 = np.deg2rad(lat1)
    lat2 = np.deg2rad(lat2)
    lon1 = np.deg2rad(lng1)
    lon2 = np.deg2rad(lng2)
    d_lon = lon2 - lon1     ##경도 차이
    d_lat = lat2 - lat1     ##위도 차이

    r = 6373.0                                          #지구 반지름
    # 위도끼리 계산 
    a = (np.sin(d_lat/2.0))**2
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1-a))
    langtitude_distance = r * c

    # 경도끼리 계산
    a = (np.sin(d_lon/2.0))**2
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1-a))
    longtitute_distance = r * c
    
    return langtitude_distance + longtitute_distance
    
# 역대여소 데이터에서 위도경도만 받아서, distance table을 생성하기 위한 빈 배열 생성
# 그 후 각 대여소별 manhatton distance를 도출하는 distance table 를 작성. (n*n) <구형 좌표계>
file_name = 'distance_btw_station.csv'
def find_distance_table():
    if file_name not in os.listdir():
          
        latlng_station_df= station_df[["위도","경도"]]  #<< 데이터 범위 변경
        size = len(latlng_station_df)
        distance_array = np.empty(shape=(size,size))
        # distance_array_XY = np.empty(shape=(size,size)

        for i in tqdm(range(len(latlng_station_df))):
            pt1 = latlng_station_df.iloc[i,].tolist()
            for j in range(len(latlng_station_df)):
                pt2 = latlng_station_df.iloc[j,].tolist()
            
                manh_dist=distance(pt1[0],pt1[1],pt2[0],pt2[1])*1000
                # manh_dist_XY = (abs(pt1[0]-pt2[0]) + abs(pt1[1]-pt2[1]))*6373.0
                distance_array[i,j] = round(manh_dist,2)
                # distance_array_XY[i,j] = round(manh_dist_XY,2)
                
        distance_df = pd.DataFrame(distance_array)
        distance_df.columns = station_df["보관소(대여소명)"]            
        distance_df.index = station_df["보관소(대여소명)"]

        distance_df.to_csv(file_name)

# fine_distance_table()


In [26]:
# parquet 데이터 불러오기
# 각 역간 distance 정보가 담겨있음.
distance_df = pd.read_csv(file_name,index_col=0)

In [27]:
geo_path = 'skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')

map.choropleth(
    geo_data=geo_str,
    data=sum_temp_info['거치대수'],
    columns=['구', '거치대수'],
    fill_color='YlGnBu',
    key_on='feature.properties.name')

map.save('folium_seoul.html')
webbrowser.open_new("folium_seoul.html")

NameError: name 'json' is not defined